In [1]:
import pickle
from datetime import datetime

import numpy as np
import pandas as pd
import scipy as sp

# import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import mean_squared_error, mean_absolute_error

/home/zhanglun/.pyenv/versions/anaconda3-4.1.0/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
now = datetime.now()
submit_path = now.strftime('../submits/%d-%H%M.csv')
submit_path

'../submits/09-0813.csv'

In [3]:
submit_predict_path = now.strftime('../submits/%d-%H%Mpredict.csv')

In [4]:
model = pickle.load(open('../dataset/lightgbmmodel.pkl', 'rb'))

In [5]:
old_model = pickle.load(open('../dataset/old_lightgbmmodel.pkl', 'rb'))

In [6]:
test_feature = pd.read_pickle('../dataset/test/all_feature.pkl')
test_feature.shape

(978298, 493)

In [7]:
use_columns = [
    x for x in test_feature.columns
    if x not in [
        'orderid', 'uid', 'hotelid', 'basicroomid', 'hotel_roomid', 'roomid', 'orderlabel'
    ]
]

In [8]:
old_result = old_model.predict_proba(test_feature[use_columns])[:, 1]

test_feature['predict'] = old_result

In [9]:
result = model.predict_proba(test_feature[use_columns+['predict']])[:, 1]

In [10]:
test_feature['prob'] = result

In [11]:
test_feature[['orderid', 'roomid', 'prob']].to_csv(submit_predict_path, index=None)

In [12]:
test_feature.sort_values('prob', ascending=False, inplace=True)

In [13]:
submit = test_feature.drop_duplicates(['orderid'])[['orderid', 'roomid', 'prob']]

In [14]:
submit['orderid'] = 'ORDER_' + submit['orderid'].astype(str)

In [15]:
submit['predict_roomid'] = 'ROOM_' + submit.roomid.astype(str)

In [16]:
submit.to_csv(submit_path, index=False, columns=['orderid', 'predict_roomid'])

In [17]:
!wc -l $submit_path

34633 ../submits/09-0813.csv


In [18]:
!head $submit_path

orderid,predict_roomid
ORDER_124079,ROOM_7000913
ORDER_131809,ROOM_3609423
ORDER_110920,ROOM_6554935
ORDER_116060,ROOM_11312703
ORDER_131199,ROOM_5334185
ORDER_135953,ROOM_11256623
ORDER_138092,ROOM_11342453
ORDER_109283,ROOM_9305729
ORDER_130155,ROOM_7740855


In [19]:
# test_feature.orderid.unique().shape